# 1. EDA 및 시각화 심화

## 1.1. 실습 개요

이 노트북에서는 고속도로 사고 데이터를 기반으로 **심화된 EDA(탐색적 데이터 분석)** 와 **Feature Engineering(파생변수 생성)** 을 수행합니다.

## 1.2. 학습 목표

1. **파생변수 생성**: 사고심각도, 요일, 주말여부 등 분석에 유용한 변수 만들기
2. **다차원 EDA**: 노선별, 시간대별, 요일별, 원인별 사고 패턴 분석
3. **피벗 테이블 활용**: 두 개 이상의 변수를 교차 분석
4. **데이터 기반 의사결정**: 분석 결과를 정책/운영 개선에 활용하는 방법 학습

## 1.3. 분석 구성

| 분석 영역 | 내용 |
|-----------|------|
| A. Hotspot 분석 | 노선별 사고 건수, 사고발생이정 분포 |
| B. 시간대 분석 | 시간대별 사고 건수, 시간대 × 원인 추이 |
| C. 요일/월별 분석 | 요일별, 평일/주말, 월별 패턴 |
| D. 원인 분석 | 원인 분포, 원인별 심각도 |
| E. 심각도 분석 | 심각도 분포, 노선별/시간대별 심각도 |

## 1.4. 사전 준비

- 이전 노트북에서 생성한 `highway_accident_preprocessed.csv` 파일 업로드

In [ ]:
# ============================================
# 2. 한글 폰트 설정 (코랩 전용)
# ============================================
# - 이전 노트북에서 설정했더라도 세션이 바뀌면 재설정 필요
# - 참조: https://conding-note.tistory.com/335

!apt-get update -qq
!apt-get install -y fonts-nanum

import matplotlib as mpl
import shutil

# matplotlib 기본 폰트를 나눔고딕으로 교체
root = mpl.matplotlib_fname().replace("matplotlibrc", "")
target_font = root + "fonts/ttf/DejaVuSans.ttf"
nanum_font = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
shutil.copyfile(nanum_font, target_font)

# 캐시 삭제
!rm -rf ~/.cache/matplotlib

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# 한글 출력 테스트
plt.figure(figsize=(4, 3))
plt.title("한글 폰트 정상 출력 확인")
plt.xlabel("가로축")
plt.ylabel("세로축")
plt.plot([1, 2, 3], [1, 4, 2])
plt.show()

In [ ]:
# ============================================
# 3. 데이터 로드
# ============================================

import pandas as pd

# - 이전 노트북(1-2)에서 저장한 전처리된 CSV 파일 로드
# - 사고년도, 사고월, 사고일, 사고시, 사고분 컬럼이 이미 분리되어 있음
file_path = "highway_accident_preprocessed.csv"
df = pd.read_csv(file_path)

print("데이터 크기:", df.shape)
display(df.head())

In [ ]:
# ============================================
# 4. 파생변수 생성 (Feature Engineering)
# ============================================

# --------------------------------------------
# 4.1. 사고일자 복원 및 요일 추출
# --------------------------------------------
# - 사고년도, 사고월, 사고일을 합쳐 사고일자 생성
# - dt.weekday: 요일 번호 반환 (월=0, 화=1, ..., 일=6)
date_str = (
    df["사고년도"].astype(str) + "-" + 
    df["사고월"].astype(str) + "-" + 
    df["사고일"].astype(str)
)
df["사고일자"] = pd.to_datetime(date_str, format="%Y-%m-%d", errors="coerce")
df["요일_번호"] = df["사고일자"].dt.weekday

# 요일 번호를 한글로 변환
weekday_map = {0: "월", 1: "화", 2: "수", 3: "목", 4: "금", 5: "토", 6: "일"}
df["요일"] = df["요일_번호"].map(weekday_map)

# --------------------------------------------
# 4.2. 주말 여부 파생변수
# --------------------------------------------
# - 토요일(5), 일요일(6)이면 1, 아니면 0
df["is_weekend"] = df["요일_번호"].isin([5, 6]).astype(int)

# --------------------------------------------
# 4.3. 사고심각도 파생변수
# --------------------------------------------
# - 사망자는 부상자보다 3배 가중치 부여
# - 사고심각도 = 사망 × 3 + 부상
df["사고심각도"] = df["사망"] * 3 + df["부상"]

# --------------------------------------------
# 4.4. 월 컬럼 정수형 변환
# --------------------------------------------
df["월"] = df["사고월"].astype(int)
df["사고시"] = df["사고시"].astype(int)
df["사고분"] = df["사고분"].astype(int)

print("파생변수 생성 완료")
display(df[["사고일자", "요일", "월", "사고시", "is_weekend", "사고심각도"]].head())

In [ ]:
# ============================================
# 5. [A] 고속도로 사고 Hotspot 분석
# ============================================

print("\n" + "=" * 60)
print("A. 고속도로 사고 Hotspot 분석")
print("=" * 60)

# --------------------------------------------
# 5.1. 노선별 사고 건수 Top 10
# --------------------------------------------
# - value_counts(): 각 값의 빈도수 계산 (내림차순 정렬)
route_counts = df["노선명"].value_counts().head(10)
print("\n[노선별 사고 건수 Top 10]")
print(route_counts)

plt.figure(figsize=(10, 5))
route_counts.plot(kind="bar", color="steelblue")
plt.title("노선별 사고 건수 (Top 10)")
plt.xlabel("노선명")
plt.ylabel("사고 건수")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --------------------------------------------
# 5.2. 상위 노선들의 사고발생이정 분포
# --------------------------------------------
# - 각 노선에서 어느 구간(km)에 사고가 집중되는지 확인
top_routes = route_counts.index.tolist()
df_top_routes = df[df["노선명"].isin(top_routes)]

plt.figure(figsize=(10, 6))
for route in top_routes:
    subset = df_top_routes[df_top_routes["노선명"] == route]
    plt.hist(subset["사고발생이정"], bins=30, alpha=0.4, label=route)

plt.title("상위 노선들의 사고발생이정(km) 분포")
plt.xlabel("사고발생이정 (km)")
plt.ylabel("사고 건수")
plt.legend()
plt.tight_layout()
plt.show()

# 5.3. Hotspot 분석 인사이트

**주요 발견:**
- 경부선, 서해안선 등 주요 간선 도로에서 사고가 집중
- 특정 구간(km)에 사고가 밀집되는 패턴 확인

**의사결정 방향:**
- 사고 다발 노선/구간에 안내 표지판, 과속 단속 카메라 우선 설치
- 특정 km 구간에 대해 도로 개선, 방호벽 보강 검토

In [ ]:
# ============================================
# 6. [B] 시간대별 사고 위험 분석
# ============================================

print("\n" + "=" * 60)
print("B. 시간대별 사고 위험 분석")
print("=" * 60)

# --------------------------------------------
# 6.1. 시간대별 사고 건수
# --------------------------------------------
hour_counts = df["사고시"].value_counts().sort_index()
print("\n[시간대별 사고 건수]")
print(hour_counts)

plt.figure(figsize=(10, 5))
hour_counts.plot(kind="bar", color="coral")
plt.title("시간대별 사고 건수")
plt.xlabel("사고 시각 (시)")
plt.ylabel("사고 건수")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# --------------------------------------------
# 6.2. 주요 원인 Top 5 추출
# --------------------------------------------
cause_counts = df["원인"].value_counts()
top_causes = cause_counts.head(5).index.tolist()
df_top_cause = df[df["원인"].isin(top_causes)]

# --------------------------------------------
# 6.3. 시간대 × 원인 피벗 테이블
# --------------------------------------------
# - pd.pivot_table(): 행/열 기준으로 집계 (엑셀 피벗과 유사)
# - index: 행 인덱스, columns: 열, values: 집계할 값, aggfunc: 집계 함수
pivot_hour_cause = pd.pivot_table(
    df_top_cause,
    index="사고시",
    columns="원인",
    values="사고년도",
    aggfunc="count"
).fillna(0)

print("\n[시간대 × 원인 피벗 테이블 (상위 5행)]")
print(pivot_hour_cause.head())

# 시간대별 원인 추이 시각화
plt.figure(figsize=(10, 6))
for cause in pivot_hour_cause.columns:
    plt.plot(pivot_hour_cause.index, pivot_hour_cause[cause], marker="o", label=cause)

plt.title("시간대별 주요 사고 원인 추이")
plt.xlabel("사고 시각 (시)")
plt.ylabel("사고 건수")
plt.xticks(pivot_hour_cause.index)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# 7. [C] 요일 및 월별 사고 패턴 분석
# ============================================

print("\n" + "=" * 60)
print("C. 요일 및 월별 사고 패턴 분석")
print("=" * 60)

# --------------------------------------------
# 7.1. 요일별 사고 건수
# --------------------------------------------
weekday_counts = df["요일"].value_counts()
print("\n[요일별 사고 건수]")
print(weekday_counts)

plt.figure(figsize=(9, 5))
weekday_counts.plot(kind="bar", color="mediumpurple")
plt.title("요일별 사고 건수")
plt.xlabel("요일")
plt.ylabel("사고 건수")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# --------------------------------------------
# 7.2. 평일 vs 주말 비교
# --------------------------------------------
weekend_counts = df["is_weekend"].value_counts().sort_index()
weekend_counts.index = ["평일", "주말"]

print("\n[평일 vs 주말 사고 건수]")
print(weekend_counts)

plt.figure(figsize=(6, 4))
weekend_counts.plot(kind="bar", color=["steelblue", "coral"])
plt.title("평일 vs 주말 사고 건수")
plt.xlabel("구분")
plt.ylabel("사고 건수")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# --------------------------------------------
# 7.3. 월별 사고 건수
# --------------------------------------------
month_counts = df["월"].value_counts().sort_index()
print("\n[월별 사고 건수]")
print(month_counts)

plt.figure(figsize=(10, 4))
month_counts.plot(kind="bar", color="teal")
plt.title("월별 사고 건수")
plt.xlabel("월")
plt.ylabel("사고 건수")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# 8. [D] 사고 원인별 특성 분석
# ============================================

print("\n" + "=" * 60)
print("D. 사고 원인별 특성 분석")
print("=" * 60)

# --------------------------------------------
# 8.1. 사고 원인 분포
# --------------------------------------------
print("\n[사고 원인 분포]")
print(cause_counts)

plt.figure(figsize=(10, 4))
cause_counts.plot(kind="bar", color="darkorange")
plt.title("사고 원인 분포")
plt.xlabel("원인")
plt.ylabel("사고 건수")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --------------------------------------------
# 8.2. 주요 원인별 평균 사고심각도
# --------------------------------------------
# - groupby(): 특정 컬럼 기준으로 그룹화하여 집계
severity_mean = (
    df_top_cause
    .groupby("원인")["사고심각도"]
    .mean()
    .sort_values(ascending=False)
)

print("\n[주요 원인별 평균 사고심각도]")
print(severity_mean.round(2))

plt.figure(figsize=(8, 4))
severity_mean.plot(kind="bar", color="crimson")
plt.title("주요 원인별 평균 사고심각도")
plt.xlabel("원인")
plt.ylabel("평균 사고심각도")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --------------------------------------------
# 8.3. 시간대별 주요 원인 분포
# --------------------------------------------
pivot_cause_hour = pd.pivot_table(
    df_top_cause,
    index="사고시",
    columns="원인",
    values="사고년도",
    aggfunc="count"
).fillna(0)

plt.figure(figsize=(10, 6))
for col in pivot_cause_hour.columns:
    plt.plot(pivot_cause_hour.index, pivot_cause_hour[col], marker="o", label=col)

plt.title("시간대별 주요 사고 원인 분포")
plt.xlabel("사고 시각 (시)")
plt.ylabel("사고 건수")
plt.xticks(pivot_cause_hour.index)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# 9. [E] 사고 심각도 분석
# ============================================

print("\n" + "=" * 60)
print("E. 사고 심각도 분석")
print("=" * 60)

# --------------------------------------------
# 9.1. 사고심각도 분포 및 기술 통계
# --------------------------------------------
print("\n[사고심각도 기술 통계]")
print(df["사고심각도"].describe())

plt.figure(figsize=(8, 4))
plt.hist(df["사고심각도"], bins=20, color="salmon", edgecolor="black")
plt.title("사고심각도 분포 (사망×3 + 부상)")
plt.xlabel("사고심각도")
plt.ylabel("사고 건수")
plt.tight_layout()
plt.show()

# --------------------------------------------
# 9.2. 노선별 평균 사고심각도 Top 10
# --------------------------------------------
severity_by_route = (
    df.groupby("노선명")["사고심각도"]
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

print("\n[노선별 평균 사고심각도 Top 10]")
print(severity_by_route.round(2))

plt.figure(figsize=(10, 5))
severity_by_route.plot(kind="bar", color="darkred")
plt.title("노선별 평균 사고심각도 (Top 10)")
plt.xlabel("노선명")
plt.ylabel("평균 사고심각도")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --------------------------------------------
# 9.3. 시간대별 평균 사고심각도
# --------------------------------------------
severity_by_hour = (
    df.groupby("사고시")["사고심각도"]
    .mean()
    .sort_values(ascending=False)
)

print("\n[시간대별 평균 사고심각도]")
print(severity_by_hour.round(2))

plt.figure(figsize=(10, 4))
severity_by_hour.sort_index().plot(kind="bar", color="navy")
plt.title("시간대별 평균 사고심각도")
plt.xlabel("사고 시각 (시)")
plt.ylabel("평균 사고심각도")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# 10. 현재 데이터 확인
# ============================================

print("최종 데이터 크기:", df.shape)
print("컬럼 목록:", df.columns.tolist())
df.head()

In [ ]:
# ============================================
# 11. Feature Engineering 완료 데이터 저장
# ============================================

# - 다음 노트북(모델링)에서 사용할 수 있도록 저장
csv_path = "/content/highway_accident_fe.csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"저장 완료: {csv_path}")

# 12. 실습 정리: 데이터 기반 의사결정 방향

## 12.1. 분석 결과 요약

| 분석 영역 | 주요 발견 |
|-----------|-----------|
| A. Hotspot | 경부선, 서해안선 등 주요 간선에 사고 집중 |
| B. 시간대 | 출퇴근 시간대(8-9시, 17-18시) 사고 다발 |
| C. 요일/월 | 평일 > 주말, 특정 월(휴가철) 사고 증가 |
| D. 원인 | 주시태만, 졸음, 과속 순으로 빈발 |
| E. 심각도 | 야간/새벽 시간대 사고 심각도 높음 |

## 12.2. 데이터 기반 의사결정 방향

| 분석 영역 | 의사결정 예시 |
|-----------|---------------|
| 노선·구간 | 사고 다발 구간에 안내 표지판, 과속 단속 카메라 우선 설치 |
| 시간대·요일 | 야간/새벽 시간대 순찰 인력 집중 배치, 휴가철 안전 캠페인 강화 |
| 사고 원인 | 졸음 사고 다발 구간에 졸음쉼터 안내 강화, 과속 다발 구간에 구간 단속 도입 |
| 심각도 | 심각도가 높은 노선/시간대에 집중적인 안전 대책 수립 |

## 12.3. 다음 단계: 예측 모델링

다음 노트북 `2_2_사고심각도_예측_모델_구축.ipynb`에서는:

1. 사고심각도를 타깃으로 한 회귀 모델 구축
2. 4가지 모델 비교 (Linear, RandomForest, XGBoost, LightGBM)
3. Feature Importance 분석을 통한 주요 변수 탐색